In [1]:
import json
import subprocess

import duckdb
import polars as pl

In [2]:
import os
import sys
import django

# Add your project root to the Python path if necessary
sys.path.append('/home/reepoi/GitHub/Orange-Button-ChecklistRegistry') 

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
# Set the DJANGO_SETTINGS_MODULE environment variable
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'checklist.settings')

# Initialize Django
django.setup()

In [3]:
from server import models

In [4]:
answer_options = pl.read_csv('ExampleChecklistAnswerOptions.csv')
print(answer_options)
answer_options = {
    col.name: [v for v in col if v is not None]
    for col in answer_options
}
answer_options['Yes / No'] = ['Yes', 'No']
answer_options['Yes/No/NA'] = ['Yes', 'No', 'NA']
answer_options['Yes / No / Unknown'] = ['Yes', 'No', 'Unknown']
answer_options = {
    k: [
        models.AnswerOption.objects.get_or_create(Value=vv)[0] for vv in v
    ]
    for k, v in answer_options.items()
}
answer_options

shape: (18, 16)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Home      ┆ Roof      ┆ Panel Man ┆ Main      ┆ … ┆ DC Wire   ┆ All       ┆ All       ┆ Monitori │
│ Style     ┆ Material  ┆ ufacturer ┆ Breaker   ┆   ┆ Run to    ┆ Labels    ┆ Strings   ┆ ng Set   │
│ ---       ┆ ---       ┆ ---       ┆ Rating    ┆   ┆ Roof      ┆ Placed    ┆ Working   ┆ Up       │
│ str       ┆ str       ┆ str       ┆ ---       ┆   ┆ J-Box     ┆ List      ┆ List      ┆ Tested   │
│           ┆           ┆           ┆ str       ┆   ┆ List      ┆ ---       ┆ ---       ┆ List     │
│           ┆           ┆           ┆           ┆   ┆ ---       ┆ str       ┆ str       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆ str       ┆           ┆           ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ Single    ┆ Cemented- ┆ Siemens/M ┆ 225A      ┆ … ┆ Short     ┆ No electr

{'Home Style': [<AnswerOption: Single>,
  <AnswerOption: 2 Story>,
  <AnswerOption: 3 Story>,
  <AnswerOption: Mobile>,
  <AnswerOption: Condo/Townhome>],
 'Roof Material': [<AnswerOption: Cemented-Down Tile>,
  <AnswerOption: Clay tile>,
  <AnswerOption: Comp Shingle>,
  <AnswerOption: Concrete>,
  <AnswerOption: Corrugated Metal>,
  <AnswerOption: Elastomeric>,
  <AnswerOption: Flat Tile>,
  <AnswerOption: Foam>,
  <AnswerOption: Gravel>,
  <AnswerOption: Metal Tile>,
  <AnswerOption: Not Applicable (No modules on the roof)>,
  <AnswerOption: Rolled Asphalt>,
  <AnswerOption: S-Tile>,
  <AnswerOption: Standing Seam Metal>,
  <AnswerOption: TPO>,
  <AnswerOption: Unable To Determine>,
  <AnswerOption: W-Tile>,
  <AnswerOption: Wood shake>],
 'Panel Manufacturer': [<AnswerOption: Siemens/Murray/ITE/Gould/Crouse Hinds>,
  <AnswerOption: Square D HOM>,
  <AnswerOption: Square D QO>,
  <AnswerOption: Eaton BH/Cutler Hammer/Westinghouse>,
  <AnswerOption: Eaton BR/Challenger/Bryant>,
  <An

In [5]:
checklist_templates = pl.read_csv('ExampleChecklist.csv')
checklist_templates

ChecklistTemplateName,DisplaySeqNumber,SectionName,QuestionLabel,AnswerType,For Select there would be 'AnswerOption',Example,,Add sample residential install and sample site survey
str,i64,str,str,str,str,str,str,str
"""Residential Site Survey""",10,"""Ground""","""Cancelled at the Door Y/N""","""EnumeratedSingle""","""Yes / No""",null,null,null
"""Residential Site Survey""",20,"""Ground""","""Home Type""","""EnumeratedSingle""","""Home Style""",null,null,null
"""Residential Site Survey""",30,"""Roof""","""Was Layout Provided for Survey""","""EnumeratedSingle""","""Yes / No""",null,null,null
"""Residential Site Survey""",40,"""Roof""","""Panels Proposed on Additional …","""EnumeratedSingle""","""Yes / No""",null,null,null
"""Residential Site Survey""",50,"""Roof""","""Roof Material""","""EnumeratedSingle""","""Roof Material""",null,null,null
…,…,…,…,…,…,…,…,…
"""Residential Installation""",480,"""Electrical Ground""","""Was A Line Side Tap Completed""","""EnumeratedSingle""","""Yes / No""",null,null,null
"""Residential Installation""",490,"""Electrical Ground""","""Was A Derate Completed""","""EnumeratedSingle""","""Yes / No""",null,null,null
"""Residential Installation""",500,"""Electrical Ground""","""Was A Hawaiian Tie In Complete""","""EnumeratedSingle""","""Yes / No""",null,null,null


In [6]:
maintainer = models.ChecklistTemplateMaintainer(
    ChecklistTemplateMaintainerName_Value='Admin',
)
maintainer.save()

In [7]:
checklist_template_rows = models.ChecklistTemplate.objects.bulk_create([
    models.ChecklistTemplate(
        ChecklistTemplateName_Value=name,
        ChecklistTemplateMaintainer=maintainer,
    )
    for name in checklist_templates['ChecklistTemplateName'].unique()
])

In [8]:
# checklist_template_rows = models.ChecklistTemplate.objects.all()
for template in checklist_template_rows:
    questions = checklist_templates.filter(ChecklistTemplateName=template.ChecklistTemplateName_Value)
    question_rows = []
    for row in questions.iter_rows(named=True):
        question = models.Question(
            SectionName_Value=row['SectionName'],
            DisplaySeqNumber_Value=row['DisplaySeqNumber'],
            QuestionLabel_Value=row['QuestionLabel'],
            AnswerType_Value=row['AnswerType'],
        )
        question.save()
        if (question_answers := row["For Select there would be 'AnswerOption'"]) is not None:
            question.AnswerOptions.set(answer_options[question_answers])
        question_rows.append(question)
    template.Questions.set(question_rows)